# para achar o race.csv e para ver as colunas e tipo delas

In [0]:
 display(dbutils.fs.ls("file:/Workspace/Users/user-uxnxwfyqmwza@oreilly-cloudlabs.com/treinamento_formula1/landing/races.csv"))

# Leitura do arquivo CSV
df = spark.read.csv("file:/Workspace/Users/user-pjaltgjixvrz@oreilly-cloudlabs.com/treinamento_formula1/landing/races.csv", header=True, inferSchema=True)

# Exibir o DataFrame para verificar o conteúdo
df = spark.read.csv("file:/Workspace/Users/user-pjaltgjixvrz@oreilly-cloudlabs.com/treinamento_formula1/landing/races.csv", header=True, inferSchema=True)

##para renomeiar as duas colunas

In [0]:
%python
# Leitura do arquivo CSV
df = spark.read.csv(
    "/Workspace/Users/user-uxnxwfyqmwza@oreilly-cloudlabs.com/treinamento_formula1/landing/circuito.csv",
    header=True,
    inferSchema=True
)

# Renomear as colunas
df = df.withColumnRenamed("raceId", "Race_id")
df = df.withColumnRenamed("circuitId", "Circuit_id")

# Salvar o DataFrame de volta como CSV em um diretório
output_directory = "dbfs:/Workspace/Users/user-pjaltgjixvrz@oreilly-cloudlabs.com/treinamento_formula1/landing/races_output"
df.write.csv(
    output_directory,
    mode='overwrite',
    header=True
)

# Exibir o DataFrame para verificar a alteração
display(df)

## para deletar url

In [0]:
codico para deleatar o url
%python
# Excluir a coluna 'url'
df = df.drop("url")

# Exibir o DataFrame para verificar a alteração
display(df)


## para criar uma nova coluna para receber o tempo alterado

In [0]:
%python
from pyspark.sql.functions import lit, concat, to_timestamp, date_format

# Supondo que as colunas 'time' e 'date' existam no DataFrame
# Concatenar as colunas 'date' e 'time' e convertê-las para o formato 'yyyy-MM-dd HH:mm:ss'
df = df.withColumn("coluna do tempo", date_format(to_timestamp(concat(df["date"], lit(" "), df["time"])), "yyyy-MM-dd HH:mm:ss"))

# Exibir o DataFrame para verificar a alteração
display(df)